## Baseline Runs - Analysis

In [3]:
import polars as pl
import torch
import numpy as np
import json
import re
## LLM
from src.llms import Llama3_1_Instruct
from src.taco_evaluator import compute
from datasets import load_from_disk
import datetime
import os

from typing import List, Dict, Any, Tuple


/home/caio.rhoden/miniconda3/envs/datamodels/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
PATH  = "../../data/TACO/processed"
train = pl.read_ipc(f"{PATH}/train.feather")
train_solutions = pl.read_ipc(f"{PATH}/train_solutions.feather")
train_dict = load_from_disk("../../data/TACO/train.hf")

In [6]:
def parse_generation(generations: list, id: int, path: str):
    
    gens = []
    for i in range(len(generations)):

        code_blocks = re.findall(r'```python(.*?)```', generations[i]["generated_text"], re.DOTALL)
        extracted_code = "\n".join([block.strip() for block in code_blocks])
        gens.append(extracted_code)
    
    results = {
        "task_id": int(id),
        "output": gens
    }

    return results

In [7]:
def join_jsons(repo:str) -> List[Dict[str, Any]]:
    jsons = []
    for file in os.listdir(repo):
        if file.endswith(".json"):
            with open(f"{repo}/{file}", "r") as f:
                jsons.append(json.load(f))
    return jsons


## EASY Examples Results

In [8]:
easy_results = join_jsons("small_experiment")

In [9]:
parsed = []
for r in easy_results:
    parsed.append(parse_generation(r["generations"], r["id"], "small_experiment"))

In [10]:
selected_ids = [r["id"] for r in easy_results]
partial_train = []
for i in selected_ids:
    partial_train.append(train_dict[i])

In [11]:
json.dump(parsed, open("parsed_small_experiment.json", "w"))

In [ ]:
compute("parsed_small_experiment.json", partial_train, [1,10,100])